In [2]:
import os


import numpy as np
from collections import defaultdict

In [3]:
valid_path = 'data/20news/valid.txt.npy'

In [4]:
tmp = np.load(valid_path)

In [5]:
tmp

array([array([None, None, 41, None, None, 97, None, None, 64, None, None, None,
       None, 97, None, None, 941, None, 644, None, None, None, None, None,
       None, None, None, 49, 292, None, None, 953, None, 492, None, 1132,
       469, 276, None, None, None, 6, None, None, 41, None, 379, 117,
       None, 508, None, None, None, None, None, 97, None, None, 291, None,
       532, 391, None, None, 97, None, 57, None, None], dtype=object),
       array([None, 1145, 61, None, None, None, None, 1003, None, None, None,
       None, None, None, 101, None, 212, None, None, 1387, None, None,
       335, None, None, None, None, 335, None, 208, None, None, None, 818,
       None, None, 514, None, 485, 64, None, 775, 194, None, None, None,
       None, None, 419, None, None, None, None, 233, 38, None, None, 57,
       208, 368, 795, 138, None, 1003, 937, None, None, None, None, None,
       None], dtype=object),
       array([29, None, None, 280, 640, 781, None, None, None, None, 118, None,
  